In [1]:
# file management
import glob
import csv
import os

# data processing
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
from skimage import io
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm

# utilities
import multiprocessing as mp
mp.set_start_method('fork', force=True)
from multiprocessing import Pool
from ipywidgets import interact, FloatSlider, Layout, interactive
import random
from tqdm import tqdm
import shutil
import itertools
import cv2
from natsort import natsorted

# Set up logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [2]:
def reorgTiffsToOriginal(data_path, conditions, subconditions):
    """
    Args:
        data_path (_type_): _description_
        conditions (_type_): _description_
        subconditions (_type_): _description_
        
        
    Activate when you have your subconditions inside the conditions folder. 
    This function renames the subconditions as PosX and moves the raw data do "original" folder.
    """
    
    
    for condition in conditions:
        # Get the actual subconditions in the directory
        actual_subconditions = [name for name in os.listdir(os.path.join(data_path, condition)) if os.path.isdir(os.path.join(data_path, condition, name))]
        
        # Rename the actual subconditions to match the subconditions in your list
        for i, actual_subcondition in enumerate(sorted(actual_subconditions)):
            os.rename(os.path.join(data_path, condition, actual_subcondition), os.path.join(data_path, condition, subconditions[i]))
        
        for subcondition in subconditions:
            # Construct the path to the subcondition directory
            subcondition_path = os.path.join(data_path, condition, subcondition)
            
            # Create the path for the "original" directory within the subcondition directory
            original_dir_path = os.path.join(subcondition_path, "original")
            
            # Always create the "original" directory
            os.makedirs(original_dir_path, exist_ok=True)
            
            # Iterate over all files in the subcondition directory
            for filename in os.listdir(subcondition_path):
                # Check if the file is a .tif file
                if filename.endswith(".tif"):
                    # Construct the full path to the file
                    file_path = os.path.join(subcondition_path, filename)
                    
                    # Construct the path to move the file to
                    destination_path = os.path.join(original_dir_path, filename)
                    
                    # Move the file to the "original" directory
                    shutil.move(file_path, destination_path)
            print(f"Moved .tif files from {subcondition_path} to {original_dir_path}")


def ensure_output_dir(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

def calculate_mean_intensity(path):
    """Calculate mean intensity of an image."""
    return io.imread(path).mean()

def calculate_protein_concentration(mean_intensity, intercept, slope):
    """Calculate protein concentration in ng/ul and nM."""
    conc_ng_ul = (mean_intensity - intercept) / slope
    return conc_ng_ul

def calculate_protein_concentration_nM(conc_ng_ul, mw_kda):
    """Convert protein concentration from ng/ul to nM."""
    conc_nM = (conc_ng_ul * 1e-3) / (mw_kda * 1e3) * 1e9
    return conc_nM

def calculate_number_of_protein_molecules(protein_mass, mw_kda):
    """Calculate number of protein molecules."""
    return (protein_mass * 6e14) / (mw_kda * 1e3)

def convert_time_units(time_values_s):
    """Convert time values from seconds to minutes and hours."""
    time_values_min = time_values_s / 60
    time_values_h = time_values_s / 3600
    return time_values_s, time_values_min, time_values_h

In [10]:
def prepare_conditions(data_path, num_reps):
    # List conditions while ignoring 'output_data'
    conditions = natsorted([
        f for f in os.listdir(data_path) 
        if os.path.isdir(os.path.join(data_path, f)) and f != 'output_data'
    ])
    
    # Generate subconditions list based on num_reps
    subconditions = [f"Rep{x}" for x in range(1, num_reps + 1)]
    
    return conditions, subconditions

# Example usage
data_path = "../../data/072224-species_30C/2ultxtl-1uldna-0p5ulMT_2/"
conditions, subconditions = prepare_conditions(data_path, 1)

# Define calibration curve paths
calibration_curve_paths = sorted(glob.glob("../../data/calibration_curve/***ugml.tif"))
reorgTiffsToOriginal(data_path, conditions, subconditions)


In [8]:

def process_image(args):
    image_file, output_directory_path, channel, slope, intercept, vmax, time_interval, i, show_scalebar, min_frame, skip_frames, condition, subcondition = args
    # Read the image into a numpy array
    intensity_matrix = io.imread(image_file)

    if channel == "cy5":
        matrix_to_plot = intensity_matrix
        # Use raw intensity for cy5 channel
        label = 'Fluorescence Intensity'
    else:
        # Convert intensity values to protein concentration using the calibration curve
        matrix_to_plot = calculate_protein_concentration(intensity_matrix, slope, intercept)
        matrix_to_plot = matrix_to_plot / 27000 * 1E6 
        label = 'Protein concentration (nM)'

    # Plot the heatmap
    fig, ax = plt.subplots(figsize=(12, 12))
    im = ax.imshow(matrix_to_plot, cmap='gray', interpolation='nearest', vmin=0, vmax=vmax)

    if show_scalebar:
        plt.colorbar(im, ax=ax, label=label)
    plt.title(f"Time (min): {(i - min_frame) * time_interval * skip_frames / 60:.2f} \nTime (h): {(i - min_frame) * time_interval * skip_frames / 3600:.2f} \n{condition} - {subcondition} - {channel}", fontsize=14)
    plt.xlabel('x [µm]')
    plt.ylabel('y [µm]')
    plt.grid(True, color='#d3d3d3', linewidth=0.5, alpha=0.5)

    # Save the heatmap
    heatmap_filename = f"heatmap_frame_{i}.png"
    heatmap_path = os.path.join(output_directory_path, heatmap_filename)
    plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0.1, dpi=300)
    plt.close(fig)

def fluorescence_heatmap(data_path, conditions, subconditions, channel, time_interval_list, min_frame, max_frame, vmax, skip_frames=1, calibration_curve_paths=None, show_scalebar=True):
    """
    Reads each image as a matrix, creates, and saves a heatmap representing the normalized pixel-wise fluorescence intensity.

    Args:
    - data_path (str): Base directory where the images are stored.
    - conditions (list): List of conditions defining subdirectories within the data path.
    - subconditions (list): List of subconditions defining further subdirectories.
    - channel (str): Channel specifying the fluorescence ('cy5' or 'gfp').
    - time_interval_list (list): List of time intervals in seconds between frames for each condition.
    - min_frame (int): Minimum frame number to start processing from.
    - max_frame (int): Maximum frame number to stop processing at.
    - vmax (float): Maximum value for color scale in the heatmap.
    - skip_frames (int): Interval to skip frames (default is 1, meaning process every frame).
    - calibration_curve_paths (list): List of file paths for the calibration curve images.
    - show_scalebar (bool): Whether to show the color scale bar in the heatmap.
    """
    output_data_dir = os.path.join(data_path, "output_data", "movies")
    ensure_output_dir(output_data_dir)

    for idx, condition in enumerate(conditions):
        time_interval = time_interval_list[idx]

        for subcondition in subconditions:
            # Determine the directory paths based on the channel
            input_directory_path = os.path.join(data_path, condition, subcondition, "original")
            output_directory_path = os.path.join(output_data_dir, f"{condition}_{subcondition}_heatmaps_{channel}")

            # Create the output directory if it doesn't exist, or clear it if it does
            if os.path.exists(output_directory_path):
                shutil.rmtree(output_directory_path)
            os.makedirs(output_directory_path, exist_ok=True)

            # Get all .tif files in the folder
            image_files = sorted(glob.glob(os.path.join(input_directory_path, f"*{channel}*.tif")))[min_frame:max_frame:skip_frames]

            # Setup calibration curve for non-cy5 channels
            slope, intercept = None, None
            if channel != "cy5":
                # Calibration curve data and fit
                sample_concentration_values = [0, 2, 5, 10, 20, 40, 80, 160, 320]

                if calibration_curve_paths is None or len(calibration_curve_paths) != len(sample_concentration_values):
                    raise ValueError(f"Mismatch in lengths: {len(calibration_curve_paths)} calibration images, {len(sample_concentration_values)} sample concentrations")

                with mp.Pool(mp.cpu_count()) as pool:
                    mean_intensity_calibration = pool.map(calculate_mean_intensity, calibration_curve_paths)
                slope, intercept = np.polyfit(sample_concentration_values, mean_intensity_calibration, 1)

            # Prepare arguments for multiprocessing
            args = [(image_file, output_directory_path, channel, slope, intercept, vmax, time_interval, i, show_scalebar, min_frame, skip_frames, condition, subcondition) for i, image_file in enumerate(image_files, start=min_frame)]

            # Use multiprocessing to process images
            with mp.Pool(mp.cpu_count()) as pool:
                list(tqdm(pool.imap(process_image, args), total=len(args), desc=f"Processing {condition} - {subcondition}"))

# Example usage

channel = "gfp"
time_interval_list = [120] * len(conditions)  # time intervals in seconds between frames for each condition
min_frame = 0
max_frame = None
vmax = 500  # Set vmax based on your data's expected concentration range
skip_frames = 64


# Call the function
fluorescence_heatmap(data_path, conditions, subconditions, channel, time_interval_list, min_frame, max_frame, vmax, skip_frames, calibration_curve_paths, show_scalebar=True)

Processing TiLa - Rep1: 100%|██████████| 10/10 [00:03<00:00,  3.31it/s]


In [9]:
channel = "cy5"
time_interval_list = [120] * len(conditions)  # time intervals in seconds between frames for each condition
min_frame = 0
max_frame = None
vmax = 14E3  # Set vmax based on your data's expected concentration range
skip_frames = 64

# Call the function
fluorescence_heatmap(data_path, conditions, subconditions, channel, time_interval_list, min_frame, max_frame, vmax, skip_frames, calibration_curve_paths, show_scalebar=False)

Processing TiLa - Rep1: 100%|██████████| 10/10 [00:02<00:00,  3.35it/s]


In [ ]:
def process_video_creation(args):
    condition, subcondition, images_dir, out_path, frame_rate, max_frame = args

    image_files = natsorted(glob.glob(os.path.join(images_dir, "*.png")))

    if not image_files:
        print(f"No images found for subcondition {subcondition}.")
        return

    # Limit the number of files if max_frame is specified
    image_files = image_files[:max_frame] if max_frame is not None else image_files

    # Get the resolution of the first image (assuming all images are the same size)
    first_image = cv2.imread(image_files[0])
    video_resolution = (first_image.shape[1], first_image.shape[0])  # Width x Height

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(out_path, fourcc, frame_rate, video_resolution)

    for file in tqdm(image_files, desc=f"Creating video for {condition}", leave=False):
        img = cv2.imread(file)
        out.write(img)  # Write the image as a frame in the video

    out.release()
    print(f"Video saved to {out_path}")

def create_movies(data_path, conditions, subconditions, channel, frame_rate=30, max_frame=None):
    """
    Creates video files from heatmaps stored in the specified directory.

    Args:
    - data_path (str): Base path where the heatmaps are stored.
    - conditions (list): List of conditions defining subdirectories within the data path.
    - subconditions (list): List of subconditions defining further subdirectories.
    - channel (str): The specific channel being processed ('cy5' or 'gfp').
    - frame_rate (int): Frame rate for the output video. Defaults to 30.
    - max_frame (int, optional): Maximum number of frames to be included in the video. If None, all frames are included.
    """
    output_data_dir = os.path.join(data_path, "output_data", "movies")

    for condition in conditions:
        args_list = []
        for subcondition in subconditions:
            images_dir = os.path.join(output_data_dir, f"{condition}_{subcondition}_heatmaps_{channel}")
            video_filename = f"{condition}_{subcondition}_{channel}.avi"
            out_path = os.path.join(output_data_dir, video_filename)

            # Prepare arguments for multiprocessing
            args_list.append((condition, subcondition, images_dir, out_path, frame_rate, max_frame))

        # Use multiprocessing to process video creation for subconditions of a single condition
        with mp.Pool(mp.cpu_count()) as pool:
            list(tqdm(pool.imap(process_video_creation, args_list), total=len(args_list), desc=f"Creating videos for condition {condition}", leave=True))

# Example usage
frame_rate = 1  # frames per second

create_movies(data_path, conditions, subconditions, channel, frame_rate=frame_rate)


In [ ]:

def quantify_tiffiles(data_path, conditions, subconditions, calibration_curve_paths, mw_kda_list, droplet_volume_list, time_interval_s_list):
    """Process images to calculate protein concentration and generate plots."""
    all_data = []

    # Sort the calibration curve paths
    calibration_curve_paths = sorted(calibration_curve_paths)

    # Calibration curve data and fit
    sample_concentration_values = [0, 2, 5, 10, 20, 40, 80, 160, 320]
    with mp.Pool(mp.cpu_count()) as pool:
        mean_intensity_calibration = pool.map(calculate_mean_intensity, calibration_curve_paths)
    slope, intercept = np.polyfit(sample_concentration_values, mean_intensity_calibration, 1)

    for idx, condition in enumerate(conditions):
        # Get condition-specific parameters
        mw_kda = mw_kda_list[idx]
        droplet_volume = droplet_volume_list[idx]
        time_interval_s = time_interval_s_list[idx]

        for subcondition in subconditions:
            # Construct paths based on condition and subcondition
            pattern = os.path.join(data_path, condition, subcondition, "original", "img_*********_gfp-4x_000.tif")
            paths = sorted(glob.glob(pattern))

            if not paths:
                print(f"No image files found for condition {condition}, subcondition {subcondition}.")
                continue

            # Calculate mean intensity for samples
            with mp.Pool(mp.cpu_count()) as pool:
                mean_intensity_list = list(tqdm(pool.imap(calculate_mean_intensity, paths), total=len(paths), desc=f"Calculating intensities for {condition} - {subcondition}"))

            # Calculate protein concentrations in ng/ul
            protein_concentration_list = [calculate_protein_concentration(intensity, intercept, slope) for intensity in mean_intensity_list]

            # Convert to nM
            protein_concentration_nM_list = [calculate_protein_concentration_nM(conc_ng_ul, mw_kda) for conc_ng_ul in protein_concentration_list]

            # Normalize intensities and concentrations
            min_intensity = min(mean_intensity_list)
            mean_intensity_list = np.array(mean_intensity_list) - min_intensity
            protein_concentration_list = np.array(protein_concentration_list) - min(protein_concentration_list)
            protein_concentration_nM_list = np.array(protein_concentration_nM_list) - min(protein_concentration_nM_list)

            # Time values
            time_values_s = np.arange(len(mean_intensity_list)) * time_interval_s
            time_values_s, time_values_min, time_values_h = convert_time_units(time_values_s)
            
            df = pd.DataFrame({
                "Condition": condition,
                "Subcondition": subcondition,
                "Time_s": time_values_s,
                "Time_min": time_values_min,
                "Time_h": time_values_h,
                "Mean Intensity": mean_intensity_list,
                "Protein Concentration_ng_ul": protein_concentration_list,
                "Protein Concentration_nM": protein_concentration_nM_list
            })

            # Calculate number of protein molecules
            protein_mass_list = df["Protein Concentration_ng_ul"] * droplet_volume
            df["Number of Protein Molecules"] = [calculate_number_of_protein_molecules(mass, mw_kda) for mass in protein_mass_list]

            # Calculate rate of change of protein molecules
            t_vals = np.linspace(0, (len(df) - 1) * time_interval_s, len(df))
            dp_dt = gaussian_filter1d(np.gradient(df["Number of Protein Molecules"], t_vals), sigma=2)
            df["Rate of Change of Number of Protein Molecules (PM/s)"] = dp_dt

            # Append the data for this condition and subcondition to the list
            all_data.append(df)

    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    # Set the output directory within the data path
    output_dir = os.path.join(data_path, "output_data")
    ensure_output_dir(output_dir)

    # Save combined results to CSV
    combined_csv_path = os.path.join(output_dir, "combined_experiment.csv")
    combined_df.to_csv(combined_csv_path, index=False)

    # Plotting
    plot_results(combined_df, output_dir, sample_concentration_values, mean_intensity_calibration, slope, intercept)

    return combined_csv_path

def plot_results(df, output_dir, sample_concentration_values, mean_intensity_calibration, slope, intercept):
    """Generate plots based on the processed data."""
    # Create subdirectories for plots
    single_plot_dir = os.path.join(output_dir, "experiment_plots", "single_plots")
    combined_plot_dir = os.path.join(output_dir, "experiment_plots", "combined_plots")
    ensure_output_dir(single_plot_dir)
    ensure_output_dir(combined_plot_dir)

    # Plot calibration curve
    plt.figure(figsize=(10, 6))
    plt.plot(sample_concentration_values, mean_intensity_calibration, 'o', label='Data points', linewidth=0.75, markersize=5)
    plt.plot(sample_concentration_values, slope * np.array(sample_concentration_values) + intercept, 'r-', label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', linewidth=0.75)
    plt.title('Mean Intensity vs Protein Concentration')
    plt.xlabel('Protein Concentration (ug/ml)')
    plt.ylabel('Mean Intensity')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(single_plot_dir, 'mean_intensity_vs_protein_concentration.png'))
    plt.close()

    # Plot calibration curve (log scale)
    plt.figure(figsize=(10, 6))
    plt.plot(sample_concentration_values, mean_intensity_calibration, 'o', label='Data points', linewidth=0.75, markersize=5)
    plt.plot(sample_concentration_values, slope * np.array(sample_concentration_values) + intercept, 'r-', label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', linewidth=0.75)
    plt.title('Mean Intensity vs Protein Concentration (Log Scale)')
    plt.xlabel('Protein Concentration (ug/ml)')
    plt.ylabel('Mean Intensity')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(single_plot_dir, 'mean_intensity_vs_protein_concentration_log.png'))
    plt.close()

    # Time units and concentration units to plot
    time_units = [("Time_s", "Time_s"), ("Time_min", "Time_min"), ("Time_h", "Time_h")]
    protein_concentration_units = [
        ("Protein Concentration_ng_ul", "Protein Concentration_ng_ul"),
        ("Protein Concentration_nM", "Protein Concentration_nM"),
        ("Number of Protein Molecules", "Number of Protein Molecules")
    ]

    # Plot protein concentration over time for each time and concentration unit
    for time_unit, time_label in time_units:
        for conc_unit, conc_label in protein_concentration_units:
            # Individual plots for each condition and subcondition
            for condition in df["Condition"].unique():
                for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
                    condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
                    plt.figure(figsize=(10, 6))
                    plt.plot(condition_data[time_unit], condition_data[conc_unit], 'o-', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
                    plt.title(f'{conc_label} vs {time_label} for {condition} {subcondition}')
                    plt.xlabel(time_label)
                    plt.ylabel(conc_label)
                    plt.legend()
                    plt.grid(True)
                    plt.savefig(os.path.join(single_plot_dir, f'{condition}_{subcondition}_{conc_label}_vs_{time_label}.png'))
                    plt.close()

            # Combined plots for all conditions and subconditions
            plt.figure(figsize=(10, 6))
            for condition in df["Condition"].unique():
                for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
                    condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
                    plt.plot(condition_data[time_unit], condition_data[conc_unit], 'o-', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
            plt.title(f'Combined {conc_label} vs {time_label} for all conditions')
            plt.xlabel(time_label)
            plt.ylabel(conc_label)
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(combined_plot_dir, f'combined_{conc_label}_vs_{time_label}.png'))
            plt.close()

            # Combined plots for all conditions and subconditions (log scale)
            plt.figure(figsize=(10, 6))
            for condition in df["Condition"].unique():
                for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
                    condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
                    plt.plot(condition_data[time_unit], condition_data[conc_unit], 'o-', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
            plt.title(f'Combined {conc_label} vs {time_label} for all conditions (Log Scale)')
            plt.xlabel(time_label)
            plt.ylabel(conc_label)
            plt.yscale('log')
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(combined_plot_dir, f'combined_{conc_label}_vs_{time_label}_log.png'))
            plt.close()

    # Plot rate of change of protein molecules
    plt.figure(figsize=(10, 6))
    for condition in df["Condition"].unique():
        for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
            condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
            plt.plot(condition_data["Time_h"], condition_data["Rate of Change of Number of Protein Molecules (PM/s)"], 'o', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
    plt.title('Rate of Change of Number of Protein Molecules vs Time_h')
    plt.xlabel('Time_h')
    plt.ylabel('Rate of Change (PM/s)')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(combined_plot_dir, 'rate_of_change_of_protein_molecules_vs_time_h.png'))
    plt.close()

    # Plot rate of change of protein molecules (log scale)
    plt.figure(figsize=(10, 6))
    for condition in df["Condition"].unique():
        for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
            condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
            plt.plot(condition_data["Time_h"], condition_data["Rate of Change of Number of Protein Molecules (PM/s)"], 'o', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
    plt.title('Rate of Change of Number of Protein Molecules vs Time_h (Log Scale)')
    plt.xlabel('Time_h')
    plt.ylabel('Rate of Change (PM/s)')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(combined_plot_dir, 'rate_of_change_of_protein_molecules_vs_time_h_log.png'))
    plt.close()

# Example usage

mw_kda_list = [100] * len(conditions)
droplet_volume_list = [2] * len(conditions)
time_interval_list = [120] * len(conditions)

# Quantify tiff files
quantify_tiffiles(data_path, conditions, subconditions, calibration_curve_paths, mw_kda_list, droplet_volume_list, time_interval_list)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.stats import norm
from tqdm import tqdm
import multiprocessing as mp
import itertools
import logging

# Ensure logging captures warnings and errors
logging.basicConfig(level=logging.WARNING)

def calculate_RpD2(R_p, D, k_TX):
    """Calculate RpD2 based on the provided parameters."""
    discriminant = (R_p + D + k_TX)**2 - 4 * R_p * D
    return 1e-6 if discriminant < 0 else 0.5 * (R_p + D + k_TX - np.sqrt(discriminant))

def dPdt(T, P, Q, S, tau_0, tau_f, tau_m, K_p):
    """Differential equation for protein concentration."""
    if T > tau_0 + tau_f:
        return Q * (1 - np.exp(-(T - tau_0 - tau_f) / tau_m)) - (S * P) / (K_p + P)
    else:
        return 0

def solve_ODE(params, N_p, N_m, D, T):
    """Solve the ODE for protein concentration."""
    k_TL, k_TX, R_p, tau_m, K_TL, R, k_deg, X_p, K_p, tau_0, tau_f = params

    RpD = calculate_RpD2(R_p, D, k_TX)
    Q = (k_TL * k_TX * RpD * tau_m) / (N_p * (1 + K_TL / R) * N_m)
    S = k_deg * X_p

    P_initial = 0
    solvers = ["LSODA", "BDF", "RK45", "Radau", "DOP853", "RK23"]
    for solver in solvers:
        try:
            p = solve_ivp(dPdt, [T[0], T[-1]], [P_initial], t_eval=T, args=(Q, S, tau_0, tau_f, tau_m, K_p), method=solver, rtol=1e-6, atol=1e-8)
            if p.status == 0:
                return p.y[0]
        except (ValueError, RuntimeError) as e:
            logging.warning(f"Solver {solver} failed with error: {e}")
            continue
    raise RuntimeError("All ODE solvers failed")

def objective_function(params, N_p, N_m, D, protein):
    """Objective function for optimization."""
    T = np.linspace(0, 5000, len(protein))
    try:
        pModel = solve_ODE(params, N_p, N_m, D, T)
    except RuntimeError as e:
        logging.error(f"ODE solver failed with error: {e}")
        return np.inf
    if pModel.shape != protein.shape:
        pModel = np.resize(pModel, protein.shape)
    return np.sum((protein - pModel)**2)

def optimize_parameters(initial_guesses, N_p, N_m, D, protein):
    """Optimize parameters using the L-BFGS-B method."""
    bounds = [(0, guess * 100) for guess in initial_guesses]
    result = minimize(objective_function, initial_guesses, args=(N_p, N_m, D, protein), method='L-BFGS-B', bounds=bounds)
    return result.x

def bootstrap_helper(args):
    """Helper function for multiprocessing bootstrapping."""
    initial_guesses, N_p, N_m, D, protein, distribution, seed = args
    np.random.seed(seed)
    bounds = [(1E-9, guess * 500) for guess in initial_guesses]

    if distribution == "uniform":
        random_initial_guesses = [np.random.uniform(low, high) for low, high in bounds]
    elif distribution == "normal":
        random_initial_guesses = [np.clip(norm.rvs(loc=guess, scale=guess * 0.3), low, high) for guess, (low, high) in zip(initial_guesses, bounds)]

    optimized_params = optimize_parameters(random_initial_guesses, N_p, N_m, D, protein)
    sse = objective_function(optimized_params, N_p, N_m, D, protein)
    return optimized_params.tolist() + [sse]

def bootstrap_optimizations(initial_guesses, N_p, N_m, D, protein, num_iterations=10, distribution="uniform", save_path=None):
    """Perform bootstrapping optimizations using all available cores."""
    args_list = [(initial_guesses, N_p, N_m, D, protein, distribution, seed) for seed in range(num_iterations)]

    with mp.Pool(mp.cpu_count()) as pool:
        results = list(tqdm(pool.imap(bootstrap_helper, args_list), total=num_iterations, desc="Bootstrap Optimizations"))

    # Convert results to DataFrame and sort
    columns = ["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f", "SSE"]
    bootstrap_df = pd.DataFrame(results, columns=columns)
    bootstrap_df["N_p"] = N_p
    bootstrap_df["N_m"] = N_m
    bootstrap_df["D"] = D
    bootstrap_df_sorted = bootstrap_df.sort_values(by="SSE")

    # Remove duplicate rows
    bootstrap_df_sorted = bootstrap_df_sorted.drop_duplicates()

    # Save ranked DataFrame if save_path is provided
    if save_path:
        bootstrap_df_sorted.to_csv(save_path, index=False)

    return bootstrap_df_sorted

def ensure_output_dir(output_dir):
    """Ensure that the output directory exists."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

def run_bootstrap_analysis(data_path, conditions, subconditions, initial_guesses_list, N_p_list, N_m_list, D_list, num_iterations=10, distribution="uniform"):
    """Run bootstrapping analysis for each condition and subcondition."""
    combined_csv_path = os.path.join(data_path, "output_data", "combined_experiment.csv")
    combined_df = pd.read_csv(combined_csv_path)

    for idx, condition in enumerate(conditions):
        initial_guesses = initial_guesses_list[idx]
        N_p = N_p_list[idx]
        N_m = N_m_list[idx]
        D = D_list[idx]

        for subcondition in subconditions:
            condition_data = combined_df[(combined_df["Condition"] == condition) & (combined_df["Subcondition"] == subcondition)]
            protein = condition_data["Protein Concentration_nM"].values

            output_dir = os.path.join(data_path, "output_data", f'{condition}_{subcondition}')
            ensure_output_dir(output_dir)
            save_path = os.path.join(output_dir, 'bootstrap_results.csv')

            bootstrap_df_sorted = bootstrap_optimizations(initial_guesses, N_p, N_m, D, protein, num_iterations, distribution, save_path)
            logging.info(f"Bootstrap results saved for condition: {condition}, subcondition: {subcondition}")

def save_theoretical_curves(bootstrap_df_sorted, protein, N_p, N_m, D, output_dir, discard_repeats=True):
    """Save theoretical curves."""
    if discard_repeats:
        bootstrap_df_sorted = bootstrap_df_sorted.drop_duplicates(subset=["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"])

    ensure_output_dir(output_dir)

    all_params = bootstrap_df_sorted[["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"]].values
    T = np.linspace(0, 5000, len(protein))

    all_params = all_params[:int(0.1 * len(all_params))]

    for rank, params in enumerate(all_params, start=1):
        theoretical_curve = solve_ODE(params, N_p, N_m, D, T)

        plt.figure(figsize=(10, 6))
        plt.plot(T, protein, label='Experimental Curve', linestyle='--', color='orange')
        plt.plot(T, theoretical_curve, label='Theoretical Curve', color='blue')
        plt.title(f'Protein Concentration vs. Time (Rank {rank})')
        plt.xlabel('Time (seconds)')
        plt.ylabel('Protein Concentration (nM)')
        plt.legend()
        plt.grid(True)
        plt.ylim(0, 1000)

        param_legend = '\n'.join([f"{name}={value:.2f}" for name, value in zip(
            ["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"], params)])
        plt.annotate(param_legend, xy=(0.05, 0.95), xycoords='axes fraction', fontsize=10, verticalalignment='top')

        plt.savefig(os.path.join(output_dir, f'theoretical_curve_rank_{rank}.png'))
        plt.close()

def plot_experimental_vs_theoretical(bootstrap_df_sorted, protein, N_p, N_m, D, output_dir, discard_repeats=True):
    """Plot experimental vs. theoretical protein concentration curves."""
    if discard_repeats:
        bootstrap_df_sorted = bootstrap_df_sorted.drop_duplicates(subset=["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"])

    ensure_output_dir(output_dir)

    all_params = bootstrap_df_sorted[["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"]].values
    T = np.linspace(0, 5000, len(protein))

    plt.figure(figsize=(10, 6))
    plt.plot(T, protein, label='Experimental Curve', linestyle='--', color='orange')

    colors = plt.cm.viridis(np.linspace(0, 1, len(all_params)))
    for rank, (params, color) in enumerate(zip(all_params, colors), start=1):
        theoretical_curve = solve_ODE(params, N_p, N_m, D, T)
        plt.plot(T, theoretical_curve, alpha=0.6, color=color, label=f'Rank {rank}')

    plt.title('Protein Concentration vs. Time')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Protein Concentration (nM)')
    plt.grid(True)
    plt.ylim(0, 1000)
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'experimental_vs_theoretical.png'))
    plt.close()

def plot_parameter_spread(bootstrap_df_sorted, output_dir, discard_repeats=True):
    """Plot the spread of optimized parameters."""
    if discard_repeats:
        bootstrap_df_sorted = bootstrap_df_sorted.drop_duplicates(subset=["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"])

    ensure_output_dir(output_dir)

    params = ["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"]
    fig, axs = plt.subplots(3, 4, figsize=(20, 15))

    colors = plt.cm.viridis(np.linspace(0, 1, len(bootstrap_df_sorted)))

    bootstrap_df_sorted = bootstrap_df_sorted.sort_values(by="SSE")

    for i, param in enumerate(params):
        ax = axs[i // 4, i % 4]
        y = bootstrap_df_sorted[param]
        x = np.arange(1, len(y) + 1) + np.random.uniform(-0.2, 0.2, size=len(y))
        c = [colors[rank] for rank in range(len(bootstrap_df_sorted))]

        ax.scatter(x, y, color=c, alpha=0.5, s=50)
        ax.set_xticks(range(1, len(bootstrap_df_sorted) + 1))
        ax.set_xticklabels(range(1, len(bootstrap_df_sorted) + 1))
        ax.set_xlabel('Rank')
        ax.set_ylabel(param)
        ax.set_title(param)

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'parameter_spread.png'))
    plt.close()

def plot_parameter_combinations(bootstrap_df_sorted, output_dir, discard_repeats=True):
    """Plot combinations of parameters."""
    if discard_repeats:
        bootstrap_df_sorted = bootstrap_df_sorted.drop_duplicates(subset=["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"])

    ensure_output_dir(output_dir)

    params = ["k_TL", "k_TX", "R_p", "tau_m", "K_TL", "R", "k_deg", "X_p", "K_p", "tau_0", "tau_f"]
    combinations = list(itertools.combinations(params, 2))

    colors = plt.cm.viridis(np.linspace(0, 1, len(bootstrap_df_sorted)))

    bootstrap_df_sorted = bootstrap_df_sorted.sort_values(by="SSE")

    for i, (param1, param2) in enumerate(combinations):
        plt.figure(figsize=(10, 6))
        color = colors[i % len(colors)]
        plt.scatter(bootstrap_df_sorted[param1], bootstrap_df_sorted[param2], alpha=0.6, color=color)
        plt.xlabel(param1)
        plt.ylabel(param2)
        plt.title(f'{param1} vs {param2}')
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, f'{param1}_vs_{param2}.png'))
        plt.close()

def run_all_plots(data_path, conditions, subconditions, initial_guesses_list, N_p_list, N_m_list, D_list, discard_repeats=True):
    """Run all plotting functions for each condition and subcondition."""
    combined_csv_path = os.path.join(data_path, "output_data", "combined_experiment.csv")
    combined_df = pd.read_csv(combined_csv_path)

    for idx, condition in enumerate(conditions):
        initial_guesses = initial_guesses_list[idx]
        N_p = N_p_list[idx]
        N_m = N_m_list[idx]
        D = D_list[idx]

        for subcondition in subconditions:
            condition_data = combined_df[(combined_df["Condition"] == condition) & (combined_df["Subcondition"] == subcondition)]
            protein = condition_data["Protein Concentration_nM"].values

            output_dir = os.path.join(data_path, "output_data", f'{condition}_{subcondition}')
            csv_path = os.path.join(output_dir, 'bootstrap_results.csv')
            bootstrap_df_sorted = pd.read_csv(csv_path)

            condition_output_dir = os.path.join(output_dir)
            ensure_output_dir(condition_output_dir)

            theoretical_curves_dir = os.path.join(condition_output_dir, 'theoretical_curves')
            experimental_vs_theoretical_dir = os.path.join(condition_output_dir, 'experimental_vs_theoretical')
            parameter_spread_dir = os.path.join(condition_output_dir, 'parameter_spread')
            parameter_combinations_dir = os.path.join(condition_output_dir, 'parameter_combinations')

            save_theoretical_curves(bootstrap_df_sorted, protein, N_p, N_m, D, theoretical_curves_dir, discard_repeats)
            plot_experimental_vs_theoretical(bootstrap_df_sorted, protein, N_p, N_m, D, experimental_vs_theoretical_dir, discard_repeats)
            plot_parameter_spread(bootstrap_df_sorted, parameter_spread_dir, discard_repeats)
            plot_parameter_combinations(bootstrap_df_sorted, parameter_combinations_dir, discard_repeats)

# Initial guesses for bootstrap optimization
initial_guesses_list = [
    [10, 1, 30, 720, 5, 190, 0.01, 9, 4, 0, 300]
] * len(conditions)

N_p_list = [401] * len(conditions)
N_m_list = [2097] * len(conditions)
D_list = [100] * len(conditions)

# Run bootstrap analysis
run_bootstrap_analysis(
    data_path, conditions, subconditions, initial_guesses_list,
    N_p_list, N_m_list, D_list, num_iterations=30, distribution="normal"
)

# Run all plots
run_all_plots(
    data_path, conditions, subconditions, initial_guesses_list,
    N_p_list, N_m_list, D_list, discard_repeats=True
)
